In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

import pickle
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [5]:
import pandas as pd
url = 'nyc_taxi_trip_duration.csv'
sample_df = pd.read_csv(url)

C:\Users\user\AppData\Local\Temp\ipykernel_3236\3008489402.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(url)


In [6]:
sample_df

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,Unnamed: 11,Unnamed: 12
0,id1080784,2,29-02-2016 16:40,29-02-2016 16:47,1,-73.953918,40.778873,-73.963875,40.771164,N,400,NaN,NaN
1,id0889885,1,11-03-2016 23:35,11-03-2016 23:53,2,-73.988312,40.731743,-73.994751,40.694931,N,1100,NaN,NaN
2,id0857912,2,21-02-2016 17:59,21-02-2016 18:26,2,-73.997314,40.721458,-73.948029,40.774918,N,1635,NaN,NaN
3,id3744273,2,05-01-2016 9:44,05-01-2016 10:03,6,-73.961670,40.759720,-73.956779,40.780628,N,1141,NaN,NaN
4,id0232939,1,17-02-2016 6:42,17-02-2016 6:56,1,-74.017120,40.708469,-73.988182,40.740631,N,848,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729317,id3905982,2,21-05-2016 13:29,21-05-2016 13:34,2,-73.965919,40.789780,-73.952637,40.789181,N,296,NaN,NaN
729318,id0102861,1,22-02-2016 0:43,22-02-2016 0:48,1,-73.996666,40.737434,-74.001320,40.731911,N,315,NaN,NaN
729319,id0439699,1,15-04-2016 18:56,15-04-2016 19:08,1,-73.997849,40.761696,-74.001488,40.741207,N,673,NaN,NaN
729320,id2078912,1,19-06-2016 9:50,19-06-2016 9:58,1,-74.006706,40.708244,-74.013550,40.713814,N,447,NaN,NaN


In [7]:
sample_df["store_and_fwd_flag"].value_counts()

N    725282
Y      4040
Name: store_and_fwd_flag, dtype: int64

In [8]:
#Convert character variables to numeric
f = lambda x: 0 if x == 'N' else 1

sample_df["store_and_fwd_flag"] = sample_df["store_and_fwd_flag"].apply(lambda x: f(x))

In [9]:



#Check result
sample_df["store_and_fwd_flag"].value_counts()

0    725282
1      4040
Name: store_and_fwd_flag, dtype: int64

In [10]:
#First, convert datetime strings into datetime
sample_df["dropoff_datetime"] = pd.to_datetime(sample_df["dropoff_datetime"], format='%d-%m-%Y %H:%M')
sample_df["pickup_datetime"] = pd.to_datetime(sample_df["pickup_datetime"], format='%d-%m-%Y %H:%M')

In [11]:
#Now construct other variables, like month, date, etc.
sample_df["pickup_month"] = sample_df["pickup_datetime"].dt.month
sample_df["pickup_day"] = sample_df["pickup_datetime"].dt.day
sample_df["pickup_weekday"] = sample_df["pickup_datetime"].dt.weekday #sample_df["pickup_weekday"] = sample_df["pickup_datetime"].dt.weekday_name
sample_df["pickup_hour"] = sample_df["pickup_datetime"].dt.hour
sample_df["pickup_minute"] = sample_df["pickup_datetime"].dt.minute

In [12]:
#Get latitude and longitude differences
sample_df["latitude_difference"] = sample_df["dropoff_latitude"] - sample_df["pickup_latitude"]
sample_df["longitude_difference"] = sample_df["dropoff_longitude"] - sample_df["pickup_longitude"]

In [13]:
#Convert duration to minutes for easier interpretation
sample_df["trip_duration"] = sample_df["trip_duration"].apply(lambda x: round(x/60))

In [14]:
#Convert trip distance from longitude and latitude differences to Manhattan distance.
sample_df["trip_distance"] = 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(sample_df["latitude_difference"]) * np.pi / 180) / 2))),
                                  np.sqrt(1-(np.square(np.sin((abs(sample_df["latitude_difference"]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(sample_df["longitude_difference"]) * np.pi / 180) / 2))),
                                  np.sqrt(1-(np.square(np.sin((abs(sample_df["longitude_difference"]) * np.pi / 180) / 2)))))))

In [15]:
sample_df.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,Unnamed: 11,Unnamed: 12,pickup_month,pickup_day,pickup_weekday,pickup_hour,pickup_minute,latitude_difference,longitude_difference,trip_distance
0,id1080784,2,2016-02-29 16:40:00,2016-02-29 16:47:00,1,-73.953918,40.778873,-73.963875,40.771164,0,7,NaN,NaN,2,29,0,16,40,-0.007709,-0.009956,1.220593
1,id0889885,1,2016-03-11 23:35:00,2016-03-11 23:53:00,2,-73.988312,40.731743,-73.994751,40.694931,0,18,NaN,NaN,3,11,4,23,35,-0.036812,-0.006439,2.988357
2,id0857912,2,2016-02-21 17:59:00,2016-02-21 18:26:00,2,-73.997314,40.721458,-73.948029,40.774918,0,27,NaN,NaN,2,21,6,17,59,0.053459,0.049286,7.098995
3,id3744273,2,2016-01-05 09:44:00,2016-01-05 10:03:00,6,-73.961670,40.759720,-73.956779,40.780628,0,19,NaN,NaN,1,5,1,9,44,0.020908,0.004890,1.782524
4,id0232939,1,2016-02-17 06:42:00,2016-02-17 06:56:00,1,-74.017120,40.708469,-73.988182,40.740631,0,14,NaN,NaN,2,17,2,6,42,0.032162,0.028938,4.221601


In [16]:
X = sample_df.drop(["trip_duration", "id", "vendor_id", "pickup_datetime", "dropoff_datetime"], axis=1)
y = sample_df["trip_duration"]

In [19]:
X.dropna()
y.dropna()

0          7
1         18
2         27
3         19
4         14
          ..
729317     5
729318     5
729319    11
729320     7
729321    20
Name: trip_duration, Length: 729322, dtype: int64

In [20]:
#Split the data into training, test, and valdiation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

In [21]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
r2score=r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
print(f"MAE :- {mae}")
print(f"MSE :- {mse}")
print(f"RMSE :- {rmse}")
print(f"R2 SCORE :- {r2score}")

In [22]:
#Define evaluation metric
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

In [23]:
#XGBoost parameters
params = {
    'booster':            'gbtree',
    'objective':          'reg:linear',
    'learning_rate':      0.05,
    'max_depth':          14,
    'subsample':          0.9,
    'colsample_bytree':   0.7,
    'colsample_bylevel':  0.7,
    'silent':             1,
    'feval':              'rmsle'
}

In [24]:
nrounds = 2000

In [27]:
X_train = X_train.drop("Unnamed: 12", axis=1)
X_val = X_val.drop("Unnamed: 12", axis=1)
X_train = X_train.drop("Unnamed: 11", axis=1)
X_val = X_val.drop("Unnamed: 11", axis=1)


In [28]:
#Define train and validation sets
dtrain = xgb.DMatrix(X_train, np.log(y_train+1))
dval = xgb.DMatrix(X_val, np.log(y_val+1))

#this is for tracking the error
watchlist = [(dval, 'eval'), (dtrain, 'train')]

In [29]:
#Train model
gbm = xgb.train(params,
                dtrain,
                num_boost_round = nrounds,
                evals = watchlist,
                verbose_eval = True
                )

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:18:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:18:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "feval", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	eval-rmse:0.67582	train-rmse:0.67384
[1]	eval-rmse:0.65407	train-rmse:0.65083
[2]	eval-rmse:0.63420	train-rmse:0.62982
[3]	eval-rmse:0.61454	train-rmse:0.60950
[4]	eval-rmse:0.59623	train-rmse:0.59023
[5]	eval-rmse:0.58343	train-rmse:0.57661
[6]	eval-rmse:0.56726	train-rmse:0.55977
[7]	eval-rmse:0.55702	train-rmse:0.54878
[8]	eval-rmse:0.54257	train-rmse:0.53305
[9]	eval-rmse:0.52974	train-rmse:0.51873
[10]	eval-rmse:0.51772	train-rmse:0.50538
[11]	eval-rmse:0.50731	train-rmse:0.49414
[12]	eval-rmse:0.49808	train-rmse:0.48402
[13]	eval-rmse:0.48754	train-rmse:0.47233
[14]	eval-rmse:0.47773	train-rmse:0.46108
[15]	eval-rmse:0.47194	train-rmse:0.45445
[16]	eval-rmse:0.46349	train-rmse:0.44436
[17]	eval-rmse:0.45734	train-rmse:0.43714
[18]	eval-rmse:0.44946	train-rmse:0.42742
[19]	eval-rmse:0.44293	train-rmse:0.41993
[20]	eval-rmse:0.43698	train-rmse:0.41239
[21]	eval-rmse:0.43099	train-rmse:0.40544
[22]	eval-rmse:0.42532	train-rmse:0.39826
[23]	eval-rmse:0.42056	train-rmse:0.39214
[2

In [31]:
X_test = X_test.drop("Unnamed: 12", axis=1)
X_test = X_test.drop("Unnamed: 11", axis=1)

In [32]:
#Test predictions
pred = np.exp(gbm.predict(xgb.DMatrix(X_test))) - 1

In [33]:
r2score=r2_score(y_test, pred)
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = mse**(1/2)
print(f"MAE :- {mae}")
print(f"MSE :- {mse}")
print(f"RMSE :- {rmse}")
print(f"R2 SCORE :- {r2score}")

4.400680294961328

In [34]:
#Take a look at feature importance
feature_scores = gbm.get_fscore()
feature_scores

{'passenger_count': 457416.0,
 'pickup_longitude': 1326580.0,
 'pickup_latitude': 1251100.0,
 'dropoff_longitude': 1128312.0,
 'dropoff_latitude': 1060111.0,
 'store_and_fwd_flag': 9256.0,
 'pickup_month': 507640.0,
 'pickup_day': 848276.0,
 'pickup_weekday': 491350.0,
 'pickup_hour': 764657.0,
 'pickup_minute': 888257.0,
 'latitude_difference': 840931.0,
 'longitude_difference': 780558.0,
 'trip_distance': 770161.0}

In [35]:
#Take a look at feature importance
feature_scores = gbm.get_fscore()
feature_scores

{'passenger_count': 457416.0,
 'pickup_longitude': 1326580.0,
 'pickup_latitude': 1251100.0,
 'dropoff_longitude': 1128312.0,
 'dropoff_latitude': 1060111.0,
 'store_and_fwd_flag': 9256.0,
 'pickup_month': 507640.0,
 'pickup_day': 848276.0,
 'pickup_weekday': 491350.0,
 'pickup_hour': 764657.0,
 'pickup_minute': 888257.0,
 'latitude_difference': 840931.0,
 'longitude_difference': 780558.0,
 'trip_distance': 770161.0}

In [36]:
#This is not very telling, so let's scale the features
summ = 0
for key in feature_scores:
    summ = summ + feature_scores[key]

for key in feature_scores:
    feature_scores[key] = feature_scores[key] / summ

feature_scores

{'passenger_count': 0.04111750484623949,
 'pickup_longitude': 0.11924738001933552,
 'pickup_latitude': 0.11246242001401398,
 'dropoff_longitude': 0.10142490452469997,
 'dropoff_latitude': 0.09529425988608135,
 'store_and_fwd_flag': 0.0008320295417230544,
 'pickup_month': 0.04563218199657426,
 'pickup_day': 0.07625223547263027,
 'pickup_weekday': 0.04416786034200765,
 'pickup_hour': 0.0687356539850179,
 'pickup_minute': 0.07984616082998003,
 'latitude_difference': 0.07559198731101015,
 'longitude_difference': 0.07016500810590579,
 'trip_distance': 0.06923041312478061}

In [37]:
filename = "xgb_model.sav"
pickle.dump(gbm, open(filename, 'wb'))

In [38]:
import pandas as pd
import numpy as np
import xgboost as xgb

from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

# Open Saved Model

In [39]:
filename = "xgb_model.sav"

In [40]:
loaded_model = pickle.load(open(filename, 'rb'))

# Genetic Algorithm

In [41]:
#Sample date

date_list = [4, 6, 2016] #April 6, 2016

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [42]:
#Sample test locations

test_locations = {'L1': (40.819688, -73.915091),
                  'L2': (40.815421, -73.941761),
                  'L3': (40.764198, -73.910785),
                  'L4': (40.768790, -73.953285),
                  'L5': (40.734851, -73.952950),
                  'L6': (40.743613, -73.977998),
                  'L7': (40.745313, -73.993793),
                  'L8': (40.662713, -73.946101),
                  'L9': (40.703761, -73.886496),
                  'L10': (40.713620, -73.943076),
                  'L11': (40.725212, -73.809179)
             }

In [43]:
geolocator = Nominatim()
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

ConfigurationError: ignored

In [ ]:
addresses

['424, East 155th Street, Melrose, Bronx County, NYC, New York, 10455, United States of America',
 '137, West 136th Street, Harlem, Manhattan Community Board 10, New York County, NYC, New York, 10030, United States of America',
 '43-11, 28th Avenue, Astoria, Queens County, NYC, New York, 11103, United States of America',
 '435, East 74th Street, Upper East Side, Manhattan Community Board 8, New York County, NYC, New York, 10021, United States of America',
 '211, Freeman Street, Greenpoint, Kings County, NYC, New York, 11222, United States of America',
 '232, East 32nd Street, Murray Hill, Manhattan Community Board 6, New York County, NYC, New York, 10016, United States of America',
 '159, West 25th Street, Penn Station South, Manhattan Community Board 4, New York County, NYC, New York, 10001, United States of America',
 '486, Brooklyn Avenue, Crown Heights, BK, Kings County, NYC, New York, 11225, United States of America',
 '70-38, 67th Place, Glendale, Queens County, NYC, New York, 11

In [ ]:
test_addresses = {'L1': '424 East 155th Street NY',
                  'L2': '137 West 136th Street NY',
                  'L3': '43-11 28th Avenue NY',
                  'L4': '435 East 74th Street NY',
                  'L5': '211 Freeman Street NY',
                  'L6': '232 East 32nd Street NY',
                  'L7': '159 West 25th Street NY',
                  'L8': '486 Brooklyn Avenue NY',
                  'L9': '70-38 67th Place NY',
                  'L10': '194 Devoe Street NY',
                  'L11': '158-46 76th Avenue NY'
             }

In [ ]:
def create_guess(points):
    """
    Creates a possible path between all points, returning to the original.
    Input: List of point IDs
    """
    guess = copy(points)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)

create_guess(list(test_locations.keys()))

['L11', 'L2', 'L7', 'L1', 'L4', 'L8', 'L3', 'L9', 'L5', 'L10', 'L6', 'L11']

In [ ]:
def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['L1', 'L5', 'L8', 'L7', 'L11', 'L2', 'L6', 'L10', 'L4', 'L3', 'L9', 'L1'], ['L5', 'L6', 'L2', 'L8', 'L9', 'L7', 'L11', 'L10', 'L1', 'L4', 'L3', 'L5'], ['L11', 'L10', 'L5', 'L3', 'L9', 'L4', 'L2', 'L1', 'L6', 'L7', 'L8', 'L11'], ['L8', 'L9', 'L3', 'L7', 'L10', 'L2', 'L11', 'L5', 'L4', 'L6', 'L1', 'L8'], ['L5', 'L3', 'L11', 'L6', 'L9', 'L10', 'L4', 'L7', 'L8', 'L1', 'L2', 'L5'], ['L8', 'L9', 'L11', 'L2', 'L7', 'L1', 'L5', 'L3', 'L6', 'L4', 'L10', 'L8'], ['L6', 'L10', 'L9', 'L2', 'L5', 'L1', 'L8', 'L4', 'L3', 'L11', 'L7', 'L6'], ['L2', 'L9', 'L8', 'L4', 'L6', 'L1', 'L5', 'L10', 'L3', 'L7', 'L11', 'L2'], ['L2', 'L4', 'L9', 'L7', 'L10', 'L11', 'L3', 'L5', 'L1', 'L6', 'L8', 'L2'], ['L4', 'L3', 'L6', 'L5', 'L10', 'L8', 'L7', 'L11', 'L2', 'L9', 'L1', 'L4']]


In [ ]:
def travel_time_between_points(point1_id, point2_id, hour, date, passenger_count = 1,
                               store_and_fwd_flag = 0, pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """

    model_data = {'passenger_count': passenger_count,
                  'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'store_and_fwd_flag' : store_and_fwd_flag,
                  'pickup_month' : my_date.month,
                  'pickup_day' : my_date.day,
                  'pickup_weekday' : my_date.weekday(),
                  'pickup_hour': hour,
                  'pickup_minute' : pickup_minute,
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'trip_distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2))),
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2))),
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2)))))))
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())

    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1

    return pred[0]

In [ ]:
coordinates = test_locations

In [ ]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score.
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

[(['L1', 'L5', 'L8', 'L7', 'L11', 'L2', 'L6', 'L10', 'L4', 'L3', 'L9', 'L1'], 292.88662338256836), (['L5', 'L6', 'L2', 'L8', 'L9', 'L7', 'L11', 'L10', 'L1', 'L4', 'L3', 'L5'], 272.61923122406006), (['L11', 'L10', 'L5', 'L3', 'L9', 'L4', 'L2', 'L1', 'L6', 'L7', 'L8', 'L11'], 230.00274276733398), (['L8', 'L9', 'L3', 'L7', 'L10', 'L2', 'L11', 'L5', 'L4', 'L6', 'L1', 'L8'], 283.04906272888184), (['L5', 'L3', 'L11', 'L6', 'L9', 'L10', 'L4', 'L7', 'L8', 'L1', 'L2', 'L5'], 276.35141468048096), (['L8', 'L9', 'L11', 'L2', 'L7', 'L1', 'L5', 'L3', 'L6', 'L4', 'L10', 'L8'], 246.69790363311768), (['L6', 'L10', 'L9', 'L2', 'L5', 'L1', 'L8', 'L4', 'L3', 'L11', 'L7', 'L6'], 274.2643165588379), (['L2', 'L9', 'L8', 'L4', 'L6', 'L1', 'L5', 'L10', 'L3', 'L7', 'L11', 'L2'], 275.72143936157227), (['L2', 'L4', 'L9', 'L7', 'L10', 'L11', 'L3', 'L5', 'L1', 'L6', 'L8', 'L2'], 285.8990898132324), (['L4', 'L3', 'L6', 'L5', 'L10', 'L8', 'L7', 'L11', 'L2', 'L9', 'L1', 'L4'], 276.7117729187012)]


In [ ]:
def get_breeders_from_generation(guesses, take_best_N=10, take_random_N=5, verbose=False, mutation_rate=0.1):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]]
    best_guess = new_generation[0]

    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)

    # Second, get some random ones for genetic diversity
    for _ in range(take_random_N):
        ix = np.random.randint(len(guesses))
        new_generation.append(guesses[ix])

    # No mutations here since the order really matters.
    # If we wanted to, we could add a "swapping" mutation,
    # but in practice it doesn't seem to be necessary

    np.random.shuffle(new_generation)
    return new_generation, best_guess

def make_child(parent1, parent2):
    """
    Take some values from parent 1 and hold them in place, then merge in values
    from parent2, filling in from left to right with cities that aren't already in
    the child.
    """
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
    child = [-99 for _ in parent1]

    for ix in range(0, len(list_of_ids_for_parent1)):
        child[ix] = parent1[ix]
    for ix, gene in enumerate(child):
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    child[-1] = child[0]
    return child

def make_children(old_generation, children_per_couple=1):
    """
    Pairs parents together, and makes children for each pair.
    If there are an odd number of parent possibilities, one
    will be left out.

    Pairing happens by pairing the first and last entries.
    Then the second and second from last, and so on.
    """
    mid_point = len(old_generation)//2
    next_generation = []

    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(make_child(parent, old_generation[-ix-1]))
    return next_generation

In [ ]:
current_generation = create_generation(list(test_locations.keys()),population=500)
print_every_n_generations = 5

for i in range(100):
    if not i % print_every_n_generations:
        print("Generation %i: "%i, end='')
        print(len(current_generation))
        is_verbose = True
    else:
        is_verbose = False
    breeders, best_guess = get_breeders_from_generation(current_generation,
                                                        take_best_N=250, take_random_N=100,
                                                        verbose=is_verbose)
    current_generation = make_children(breeders, children_per_couple=3)

Generation 0: 500
['L10', 'L8', 'L7', 'L2', 'L1', 'L3', 'L11', 'L9', 'L6', 'L4', 'L5', 'L10']
Generation 5: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 10: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 15: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 20: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 25: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 30: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 35: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 40: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 45: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 50: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11

In [ ]:
def evolve_to_solve(current_generation, max_generations, take_best_N, take_random_N,
                    mutation_rate, children_per_couple, print_every_n_generations, verbose=False):
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i: "%i, end='')
            print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False
        breeders, best_guess = get_breeders_from_generation(current_generation,
                                                            take_best_N=take_best_N, take_random_N=take_random_N,
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess))
        current_generation = make_children(breeders, children_per_couple=children_per_couple)

    return fitness_tracking, best_guess

current_generation = create_generation(list(test_locations.keys()),population=500)
fitness_tracking, best_guess = evolve_to_solve(current_generation, 100, 150, 70, 0.5, 3, 5, verbose=True)

Generation 5: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 10: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 15: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 20: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 25: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 30: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 35: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 40: 330
Current Best Score:  209.986343383789